# Kerasでのネットワーク実装

手作りのニューラルネットワーク専用のフレームワーク「Keras」で構築しよう。まず、必要なパッケージを追加しよう

In [2]:
import numpy as np

# 入力、結合、活性化の「レイヤ」
from keras.layers import Input, Dense, Activation

# モデル
from keras.models import Model

# 活性化関数
from keras.activations import sigmoid

## モデル構築

Kerasのモデルは「レイヤ」で構築する。ネットワークの各パーツにたいして、適するレイヤがある：

- Input: 入力レイヤ。入力の数を定義する
- Dense: 全結合レイヤ。前レイヤの出力と本レイヤの出力をすべてつながるり、荷重を与える
- Activation：活性レイヤ。前レイヤの出力の出力に活性化関数で処理する

In [3]:
def create_model():

    # 入力レイヤ
    x = Input(shape=(2,), dtype=float)

    # 全結合レイヤ（荷重）出力が１つ
    w = Dense(units=1)(x)

    # 活性化レイヤ。活性化関数はsigmoid
    y = Activation(sigmoid)(w)

    # モデル
    model = Model(inputs=x, outputs=y)

    return model

In [4]:
# 確認しましょう
model = create_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 2)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             3 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3 (12.00 B)

 Trainable params: 3 (12.00 B)

 Non-trainable params: 0 (0.00 B)

上記の出力を解析してみると…

`input_1 (InputLayer)        [(None, 2)]               0 `

入力レイヤは入力の数は２つがある。`None`はデータの数を示している。ANDとORの課題で、データの数は４点があるが、現時点でまだ確定ではないので`None`として表示されている。入力レイヤは学習できるパラメータはないので、パラメータの数は0である。

`dense (Dense)               (None, 1)                 3`

全結合レイヤの出力は１つしかない。上記と同様、まだ確定されていないデータの数は`None`で表示。なお、パラメータの数は3となる（荷重２つとバイアス１つ）

`activation (Activation)     (None, 1)                 0`

最後に活性化関数があり、出力は１つです。前の出力を処理するだけで、学習パラメータがない。

In [5]:
# 中身を確認
print (model.get_weights())

[array([[0.44639003],
       [1.3057963 ]], dtype=float32), array([0.], dtype=float32)]


## 推論
このまま（学習なし）で推論をしてみると：

In [6]:
x = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
], dtype=float)

y_pred = model.predict(x)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step
[[0.5       ]
 [0.78680885]
 [0.60978055]
 [0.85222834]]


## 学習

推論はまったく当たらないので、学習させましょう！

まず、kerasでは、そのまま学習できない。その先、`create_model()`で作ったモデルを「コンパイル」しなければならない。コンパイルすることにて、モデルをGPUで実行できるようになる

In [7]:
model.compile(optimizer="sgd", loss="mean_squared_error")

上記の命令を解説すると：

- `optimizer`:「最適化方法」という意味であり、勾配関数のことを表している。`sgd`と`Adam`をよく使われている
- `loss`:損失関数のこと。課題により、適する損失関数がありが、とりあえず引き続きに「平均二乗誤差」を利用

なお、学習は、以下の命令で実装：

`model.fit(x, y_true, batch_size=1, epochs=1500)`

- `x`:入力データ
- `y_true`:正しい答えとして期待しているラベル
- `epochs`:学習する回数
- `batch_size`:「バッチサイズ」は、入力データを何点を一緒にまとめて学習させるのか

In [8]:
# 学習データを用意
y_true = np.array([
    [0],
    [0],
    [0],
    [1]
], dtype = float)


model.fit(x, y_true, batch_size=1, epochs=1500)

Epoch 1/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.3452 
Epoch 2/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2792 
Epoch 3/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2105 
Epoch 4/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2773 
Epoch 5/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2575 
Epoch 6/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4085 
Epoch 7/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3539 
Epoch 8/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2291 
Epoch 9/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2665 
Epoch 10/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3974 
Epoch 11/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2017 
Epoch 12/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2872 
Epoch 13/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3412 
Epoch 14/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2836 
Epoch 15/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1975 
Epo

In [9]:
#結果を確認
y_pred = model.predict(x)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
[[0.07973862]
 [0.29318842]
 [0.2703366 ]
 [0.6394643 ]]


## 改善方法：適する損失関数を選ぶ

「平均二乗誤差」普段に「距離」が表している（正解数値からどれぐらい離れるのか？）。実数の場合はこれで良いと思っていいのが、学習データはバイナリ（0か1）の場合、kerasがもと良い損失関数を提供する。それは`binary_crossentropy`という。

それでは、関数を変えて、確かめてみよう～

In [10]:
# もう一度乱数を固定、モデルを構築する
model = create_model()
model.compile(optimizer="sgd", loss="binary_crossentropy")

In [11]:
# 学習実行
model.fit(x, y_true, batch_size=1, epochs=1500)

Epoch 1/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.7885 
Epoch 2/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.8202
Epoch 3/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.8360 
Epoch 4/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.6101
Epoch 5/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6260 
Epoch 6/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7360 
Epoch 7/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7352 
Epoch 8/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6231 
Epoch 9/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7671 
Epoch 10/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6192 
Epoch 11/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5974 
Epoch 12/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.6838
Epoch 13/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.7544 
Epoch 14/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.5986
Epoch 15/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6771


In [12]:
#結果を確認
y_pred = model.predict(x)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
[[0.02246707]
 [0.19473651]
 [0.19421992]
 [0.71720505]]


In [13]:
# 加重を確認
print(model.get_weights())

[array([[2.3501616],
       [2.3534591]], dtype=float32), array([-3.7729812], dtype=float32)]


## 練習：ORを学習させる

## 練習：XORを学習させる